# Pitstops and their Impact on Race Outcome
We will be exploring pitstop data from F1 seasons 2018-2023 and looking at how they determine the outcome of the races

## STEP 1 - Loading the Data & Libraries

In [ ]:
# Let us start by importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.model_selection as model_selection

# Load the datasets
races = pd.read_csv('data/races.csv')
results = pd.read_csv('data/results.csv')
pit_stops = pd.read_csv('data/pit_stops.csv', dtype={'milliseconds': 'int'})

# Display the first few rows of the dataset
races.head(), results.head(), pit_stops.head() 

## STEP 2 - Filtering the Data
We are only interested in data from seasons 2018-2023 so let us try to filter the dataset

In [ ]:
# Filtering the dataset for years 2018 to 2023
races_2018_2023 = races[(races['year'] >= 2018) & (races['year'] <= 2023)]
# Dropping unnecessary columns
races_2018_2023.drop(columns=['fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'url'], inplace=True)

# Getting the raceId for the years 2018 to 2023
raceIds_2018_2023 = races_2018_2023['raceId'].unique()

# Filtering pit_stops & results dataset with the raceIds from 2018 to 2023
pit_stops_2018_2023 = pit_stops[pit_stops['raceId'].isin(raceIds_2018_2023)]
results_2018_2023 = results[results['raceId'].isin(raceIds_2018_2023)]

# Display the shape of the filtered datasets
races_2018_2023.shape, pit_stops_2018_2023.shape, results_2018_2023.shape


## STEP 3 - Cleaning the Datasets
Now that we have the filtered data we can carry on with cleaning the data by handling missing values and outliers

In [ ]:
# Check for missing values in the filtered datasets
missing_values_races = races_2018_2023.isnull().sum()
missing_values_results = results_2018_2023.isnull().sum()
missing_values_pit_stops = pit_stops_2018_2023.isnull().sum()

missing_values_races, missing_values_results, missing_values_pit_stops

There are no missing values, however some have been filled with '/N' indicating a null value. Normally we would have to inspect, replace or in some instances remove those from out dataset because they could skew the analysis in many ways, but in this instance we're keeping them because for the datasets in question they signify that the racer has dropped out of the race for various reasons. Alternatively we could replace them with a numerical value for further analysis such as drop out percentage etc.

Let us visualize the data in the form of histograms, boxplots and scatterplots to help further our analysis

In [ ]:
# Checking the descriptive statistics of the filtered datasets
pit_stops_2018_2023.describe(), results_2018_2023.describe()

In [ ]:

# Histogram for Lap Distribution in Pit Stops
sns.histplot(pit_stops_2018_2023['lap'], bins=150)


We can see that most drivers have a pit stop around the first two laps. This can be caused by faulty equipment, accidents or a change in strategy.

Apart from the first two laps the data shows signs of normal distribution. We can overlay the normal distribution curve to check

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

plt.figure(figsize=(10, 6))
sns.histplot(pit_stops_2018_2023['lap'], kde=False, stat="density", bins=30, color='skyblue', edgecolor='black')

# Overlay the normal distribution curve
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(pit_stops_2018_2023['lap']), np.std(pit_stops_2018_2023['lap']))
plt.plot(x, p, 'k', linewidth=2)
plt.title('Histogram of Lap Distribution in Pit Stops with Normal Distribution Curve')
plt.xlabel('Lap Number')
plt.ylabel('Density')
plt.grid(True)
plt.show()


Let us take a look at how the milliseconds, meaning the duration of the stops in ms is distributed

In [ ]:
sns.histplot(pit_stops_2018_2023['milliseconds'], bins=150)
plt.xlim([10000, 100000])

In [ ]:
sns.boxplot(x='milliseconds', data=pit_stops_2018_2023)
plt.xlim([10000, 100000])

In [ ]:
# Scatter plot for Pit Stop Duration vs Lap Number
plt.figure(figsize=(10, 6))
sns.scatterplot(data=pit_stops_2018_2023, x='lap', y='milliseconds')
plt.title('Scatter Plot of Pit Stop Duration vs Lap Number')
plt.xlabel('Lap Number')
plt.ylabel('Pit Stop Duration (milliseconds)')
plt.ylim(10000, 100000)
plt.grid(True)
plt.show()

In [ ]:
# the median pit stop duration
median_pit_stop = pit_stops_2018_2023['milliseconds'].median()
median_pit_stop

We can see from the boxplot that pitstops above 40.000 ms are considered outliers. The outliers are however better visualized in the scatterplot were we can determine that anything above 50.000 ms is a more precise way to determine what are outliers. That's a duration of 50 seconds for a pitstop, which is unusually long especially considering the median is around 23 seconds.
So let us proceed with removing pitstops above 50.000 ms 

In [ ]:
# removing outliers from the pit stop dataset anything above 50,000 milliseconds
pit_stops_2018_2023 = pit_stops_2018_2023[pit_stops_2018_2023['milliseconds'] <= 50000]

# removing time column from the pit stops dataset
pit_stops_2018_2023.drop(columns=['time'], inplace=True)

# Checking the descriptive statistics of the pit stops dataset
pit_stops_2018_2023.describe()

Now let us merge the data from pit stops and results into one dataset and choose the columns we want to investigate further to see if there is basis for making predictions on outcome or otherwise.

In [ ]:
# Merge the datasets based on raceId and driverId
merged_data = pd.merge(results_2018_2023, pit_stops_2018_2023, on=['raceId', 'driverId'])

# rename milliseconds_x to milliseconds_lap and milliseconds_y to milliseconds_pit
merged_data.rename(columns={'milliseconds_x': 'milliseconds_lap', 'milliseconds_y': 'milliseconds_pit'}, inplace=True)

# Select relevant columns for further analysis
relevant_columns = ['raceId', 'driverId', 'stop', 'lap', 'milliseconds_pit', 'positionOrder', 'points', 'laps']
merged_data = merged_data[relevant_columns]

# Display the merged dataset
merged_data.head()


In [ ]:
# Save merged dataset to a csv file
merged_data.to_csv('data/merged_data.csv', index=False)
# Descriptive statistics of the merged dataset
merged_data.describe()

To understand the relationship between the different variables we will calculate the correlation matrix and visualize it in a heatmap

In [ ]:
# Correlation matrix of the merged dataset
correlation_matrix = merged_data.corr()

# Heatmap of the correlation matrix
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap of Correlation Matrix')
plt.show()

From this we can observe a slight correlation

In [ ]:
# Relationship between pit stop duration and race position
plt.figure(figsize=(10, 6))
sns.scatterplot(x='milliseconds_pit', y='positionOrder', data=merged_data)
plt.ylim(1, 10)
plt.title('Pit Stop Duration vs. Race Position')
plt.xlabel('Pit Stop Duration (milliseconds)')
plt.ylabel('Race Position (Order)')
plt.show()

From what we can gather there is no significant correlation between pit stop duration and the outcome of the race. For exam related purpose I will try to build a predictive model to forecast the race outcome using all the data. For this I will use the Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Select features and target variable
features = merged_data[['driverId', 'stop', 'lap', 'milliseconds_pit', 'points', 'laps']]
target = merged_data['positionOrder']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict race positions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display accuracy and classification report
accuracy, report



In [23]:
# Display accuracy
accuracy

0.5924882629107981

In [25]:
# Display classification report
print(report)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        67
           2       1.00      1.00      1.00        47
           3       1.00      1.00      1.00        61
           4       1.00      1.00      1.00        55
           5       1.00      1.00      1.00        52
           6       1.00      0.98      0.99        53
           7       0.98      1.00      0.99        50
           8       1.00      0.98      0.99        65
           9       0.98      1.00      0.99        45
          10       1.00      1.00      1.00        52
          11       0.11      0.13      0.12        52
          12       0.14      0.11      0.12        64
          13       0.16      0.16      0.16        62
          14       0.15      0.15      0.15        66
          15       0.21      0.31      0.25        51
          16       0.20      0.15      0.17        61
          17       0.15      0.14      0.15        56
          18       0.13    

The classification report indicates that the model is highly precise and accurate in predicting the higher positions (positions 1 to 10), but it struggles significantly with predicting the lower positions (positions 11 to 20).